# *Introduction to few key NLP metrics*

In any scenario, where there is a need to judge how good a machine translation of a given source language is, or judge the performance of a classifier, we need to introduce metrics. Note that, in case of machine translation source and target language can be anything--a PDF, latex code corresponding to a PDF, any human language, etc. Hence these metrics are language-independent. We attempt to introduce few of these metrics here:

+ Levenshtein distance (Edit distance)
+ BLEU (BiLingual Evaluation Understudy)
+ ROUGE (Recall Oriented Understudy for Gisting Evaluation)
+ Precision
+ Accuracy
+ Recall
+ F1 Score

## Machine Translations related

### Levenshtein distance
 Levenshtein or Edit distance is a fundamental string metric. Given two strings $A$ and $B$, it computes minimum number of insertions, deletions or replacements needed to transform $A$ to $B$. It helps to compute how much different a machine translated string is from the source string. Edit distance has a convenient recursive formula:
 
 $$lev_{a,b}(i,j) = \begin{cases} 
          lev_{a,b}(i+1,j+1) & if \ a[i]=b[j], a,b\neq\phi \\
          1+min\{lev_{a,b}(i,j+1),lev_{a,b}(i+1,j),lev_{a,b}(i+1,j+1)\} & if \ a[i] \neq b[j], a,b\neq \phi \\
          |a[i:]| & if \ b=\phi \\
          |b[j:]| & if \ b\neq\phi, a=\phi
       \end{cases}$$
       
where <mark>$lev_{a,b}(i,j)$ is the levenshtein distance of the string $a$ starting from $i^{th}$ index w.r.t. string $b$ starting from $j^{th}$ index</mark>. 

*EXPLANATION*: In order to compute $lev_{a,b}(i,j)$:
+ If $a[i] = b[j]$, then it is quite clear that we recursively compute the Edit distance of the string $a[i+1:]$ w.r.t. $b[j+1:]
 $
+ If $a[i] \neq b[j]$, then either:
  + we add $b[j]$ in the current position of $a$, so we recursively compute the Edit distance of the string $a[i:]$ w.r.t. $b[j+1:]$ (as $b[j]$ has been matched but didn't exhaust any character of the original $a$) and add $1$ to result(for the $Add$ operation).
  + we replace $a[i]$ with $b[j]$, so we recursively compute the Edit distance of the string $a[i+1:]$ w.r.t. $b[j+1:]$ (since $b[j]$ has been matched, and exhausted one more character of $a$) and add $1$ to result(for the $Replace$ operation).
  + we delete $a[i]$ in hope that the rest of the $a$ string matches with rest of the $b$ string, so we recursively compute Edit distance of $a[i+1:]$ w.r.t. $b[j]$(since character of $a$ has been exhausted without matching $b[j]$)
 
The recursive formula, naturally leads to a bottom up dynamic programming approach, where we take a $(|a|+1)$x$(|b|+1)$ matrix $dp$ with rows indexed by $a$ and column by $b$ such that (we output $dp[0][0]=lev_{a,b}(0,0)$ at the end):
$$ dp[i][j] =\begin{cases}
lev_{a,b}(i,j) & if \ 0\leq i\leq|a|, 0\leq j\leq|b|\\
lev_{a,\phi}(i,j) & if \ j=|b| \\
leb_{\phi,b}(i,j) & if \ i=|a|\\
\end{cases}$$


In [30]:
# runs in O(len(str1)*len(str2))
def levenshtein( str1, str2):
    l1 = len(str1)
    l2 = len(str2)
    matrix = [[-1]*(l2+1) for i in range(0,l1+1)] # initialisation of the (len(str1)+1)x(len(str2)+1) matrix with -1(s)
    
    # base cases
    for i in range(l2+1):
        matrix[l1][i] = l2 - i # initialisation of the bottom row
    
    for j in range(l1+1):
        matrix[j][l2] = l1 - j # initialisation of the rightmost column
        
    # computation starts!
    for i in range(l1-1,-1,-1): # bottom-up fashion--- from second last row to top row, and in each row from second-last cell to starting cell
        for j in range(l2-1,-1,-1):
            if str1[i]==str2[j]:
                matrix[i][j] = matrix[i+1][j+1] # if the characters are equal, move both pointers
            else:
                matrix[i][j] = 1 + min(matrix[i+1][j],matrix[i+1][j+1], matrix[i][j+1]) # else, add 1 (pertaining to either insert, replace or delete) to the minimum of the levenshstein distances of the remaining pieces of the strings
            
    
    return matrix[0][0]



Examples:
+ levenshtein distance of *"abc"* w.r.t *"adc"* is $1$ : *"abc"*  $\xrightarrow[]{\text{replace "b" with "d"}}$ *"adc"*
+ levenshtein distance of *"horse"* w.r.t. *"ros"* is $3$ : *"horse"* $\xrightarrow[]{\text{replace "h" with "r"}}$ *"rorse"* $\xrightarrow[]{\text{delete "r"}}$ *"rose"* $\xrightarrow[]{\text{delete "e"}}$ *"ros"* 

In [31]:
print(levenshtein("abc", "adc"))
print(levenshtein("horse","ros"))

1
3


### BLEU metric



<a id='one'></a><mark>Definition 1 (*$n-gram$*)</mark>: Generally an $n-gram$ is a sequence of $n$ consecutive words, letters, syllables or base pairs (*for machine translation(MT) purposes they are generally words*) collected from a text corpus(dataset).

Ex: bigrams($2$-grams) of *"This article is on NLP"* are *"This article"*, *"article is"*, *"on NLP"*, etc. 

Given an Machine Translation of a source "sentence", and one or more "references", we need to able to say how close the translaton is w.r.t the references and clearly distinguish a good candidate from a bad one.

<mark>Definition 2(*modified $n-gram$ precision*)</mark> Modified $n-gram$ precision of a candidate sentence w.r.t. one or more references is 

$$ \frac{\sum\limits_{\mathcal{C}\in unique \  n-grams} \ count\_clip(\mathcal{C})}{number \ of \ n-grams}$$, 

where 
$count\_clip(\epsilon) = min(Count, Max\_Ref\_Count)$ , $Count$=number of times the $n-gram$ $\epsilon$ appears in the candidate sentence, $Max\_Ref\_Count$ is the maximum Count of $\epsilon$ over all reference(sentences).

We can calculate the modified $n-gram$ precision using `nltk.translate.bleu_score.modified_precision` method of the NLTK library. (install NLTK using `pip install nltk`)

In [32]:
import nltk
# example 1


reference  = '\dfrac{1}{\sqrt{n} \Sigma_{i=1}^{n} |i \rangle'
ref        = reference.split()

candidate1 = '\dfrac{1}{\sqrt{n} \Sigma\limits_{i=1}^{n} |i \rangle'
cand       = candidate1.split()
print("Number of unigrams =",len(cand))
references = [ref]
float(nltk.translate.bleu_score.modified_precision(references, cand, n=1)) # modified 1-gram precision = 37/44


Number of unigrams = 4


0.75

In [33]:
# example 2

reference1 = 'It is a guide to action that ensures that the military will forever heed Party commands'.split()
reference2 = 'It is the guiding principle which gurantees the military forces always being under the command of the Party'.split()
reference3 = 'It is the practical guide for the army always to heed the directions of the party'.split()

candidate1 = 'It is to insure the troops forever hearing the activity guidebook that party direct'.split()
references = [reference1, reference2, reference3]
print(float(nltk.translate.bleu_score.modified_precision(references, candidate1, n=2))) # modified 2-gram precision = 1/13
float(nltk.translate.bleu_score.modified_precision(references, candidate1, n=1)) # modified 1-gram precision = 8/14

0.07692307692307693


0.5714285714285714

Note: Since we calculate modified $n-gram$ precision for a "sentence"(for  $\LaTeX$ code, it might refer to a single sequence of tokens representing the candidate $\LaTeX$ code for an equation, an image, a table, etc), in order to calculate $n-gram$ precision (denoted $p_n$ )for the whole document(collection of "sentences") we use the following formula:
$$ p_n = \frac{\sum\limits_{\mathcal{C}\in \{ Candidates \}}\sum\limits_{n-gram \in \mathcal{C}}count\_clip(\mathcal{n-gram})}{\sum\limits_{\mathcal{C'}\in \{ Candidates \}}\sum\limits_{n-gram \in \mathcal{C'}}count(\mathcal{n-gram})} $$.

<table><tr><th>Why should we consider a weighted average of $n-gram$ precisions over multiple $n$?</th></tr></table>
It is obvious as $n$-increases(upto a certain point) modified $n-gram$ precision of a corpus(whole document) better relates to translation quality:

+ *low $n$*: Checks the vocabulary of tokens/words used in the candidate, i.e. ensures the candidate doesn't use too many unrelated tokens, but it doesn't enforce token order and one might have a complete garbage candidate with correct tokens all jumbled up.

+ *high $n$*: Checks and ensures proper token order and longer syntactic rules, but may be too harsh on minor deviations from "desired" references.

Hence, we need to average out the modified precision scores for all $n\leq N$(threshold).

But...
<figure>
    <img src="image1.jpg" alt="Alt text" />
    <figcaption><center><mark>Exponentially decaying $n-gram$ scores. Here $H_{i}:Human_{i} $,$S_{j}:System/Machine_{j}$</mark></center></figcaption>
</figure>

as the image shows, modified $n$-gram scores decays rough exponentially with $n$ for human translators("good") as well as machine translators("bad").

Therefore we use a heuristic wherein we take <mark>weighted average of logarithms of $p_n$</mark> for $n\leq N$. (The baseline BLEU uses uniform weights)

<mark>Sentence length</mark>: A candidate should neither be too short, nor too long, already the modified $n-gram$ precisions punish longer candidates that repeat a particular instance of a token too much, or use spurious words, **but**, too short candidates using the right words and orders(ex: a fragment of a reference) are spared, as seen in the following example.  Therefore we deploy a *brevity penalty* on longer sentences.




In [34]:
# example
reference1 = 'It is a guide to action that ensures that the military will forever heed Party commands'.split()
reference2 = 'It is the guiding principle which gurantees the military forces always being under the command of the Party'.split()
reference3 = 'It is the practical guide for the army always to heed the directions of the party'.split()

candidate1 = 'of the'.split() # very short and undesirable candidate
references = [reference1, reference2, reference3]
for i in range(1,3):
    print(float(nltk.translate.bleu_score.modified_precision(references, candidate1, n=i))) # score is still 1.0

1.0
1.0


<mark>Definition $3$(*Brevity penalty*)</mark>: Let $r_s(\mathcal{C}) = |\epsilon| s.t. ||\epsilon|-|\mathcal{C}||=\min\limits_{\mathcal{R}\in \{References\}}||\mathcal{C}|-|\mathcal{R}||$. Call $r_s$ the *best match length*.
Let $r$(*effective refernce corpus length*) be the sum of *best match lengths*($r_s$) of each sentence in the corpus, then $$BP(\text{Brevity Penalty}) =\begin{cases}
1 & if \ c > r\\
e^{1-\frac{r}{c}} & if \ c \leq  r
\end{cases}$$, where $c=|\mathcal{C}|$






In [35]:
reference1 = 'It is a guide to action that ensures that the military will forever heed Party commands'.split()
reference2 = 'It is the guiding principle which gurantees the military forces always being under the command of the Party'.split()
reference3 = 'It is the practical guide for the army always to heed the directions of the party'.split()

candidate1 = 'of the'.split() # very short and undesirable candidate
references = [reference1, reference2, reference3]
cand_len = len(candidate1)
closest_ref_len =  nltk.translate.bleu_score.closest_ref_length(references, cand_len)
nltk.translate.bleu_score.brevity_penalty(closest_ref_len, cand_len) # brevity penalty

0.0009118819655545162

Final BLEU Metric:

$$BLEU = BP. \exp{(\sum\limits_{n=1}^{N}w_n \log{p_n})}$$

In the baseline metric, $w_n=\frac{1}{N}$ and $N=4$


In [36]:
# example
reference  = '\dfrac{1}{\sqrt{n} \Sigma_{i=1}^{n} |i \rangle'
ref        = [char for char in reference]

candidate1 = '\frac{1}{\sqrt{n} \Sigma\limits_{i=1}^{n} |i>'
cand       = [char for char in candidate1]
list_of_references = [[ref]]
hypotheses = [cand]

nltk.translate.bleu_score.corpus_bleu(list_of_references, hypotheses) # BLEU score, by default uniform weight and N=4


0.7447490192819548

### ROUGE metric
 ROUGE stands for Recall-Oriented Understudy for Gisting Evaluation. It incorporates Recall and Precision(the formula "follows" from the basic one in [here](#two)) in its multiple variants.ROUGE is actually a family of four different measures: `ROUGE-N`, `ROUGE-L`, `ROUGE-W` and `ROUGE-S`. 
 
We describe three of them(`ROUGE-N`, `ROUGE-L` and `ROUGE-W`) assisted with some minimal python implementation.

### ROUGE-$N$:

For a single reference $R$ and a candidate $C$, ROUGE_$N (R,C)$= $\dfrac{ \sum\limits_{ distinct \ ng \in R,ng:  N-gram}Count_{match}(ng)}{ \sum\limits_{ng \in R,ng: N-gram}Count(ng)}$.

For the definition of $N$-gram refer [here](#one).. In the above formula, in the numerator, the summation goes over all distinct $N$-grams $ng$ in it. $Count_{match}(ng)$ refers to the maximum count of $ng$ co-occuring in the Reference as well as in the Candidate.

***What should we do for multiple references?***

We simply take the maximum over the pairwise ROUGE_$N$ scores for each reference $r_i$ and candidate $C$, i.e.,
ROUGE-$N_{multi}$ = $\max\limits_{r_i}$ ROUGE-$N$ ($r_i$,$C$).

In the python implementation, however, we do a *Jackknifing procedure*: Given $M$ references, we calculate ROUGE-$N_{multi}$ scores for each subset of $M-1$ references($M$ such choices) and take the average of the $M$ scores obtained. This fixes the problem of inflated score, where the candidate correlates highly with only a few of the references, and very poorly with others. 

We use the wonderful `rouge_score` library https://github.com/pltrdy/rouge uploaded to PYPI for implementations. (install using `pip install rouge-score`). We could also load the `rouge` module from `evaluate` library, but I figured out it uses `rouge_score` library too!)

**ALERT**: The default tokenizer used in `rouge_score` discards non-alphanumeric characters and replaces them by white-spaces, so I made my tokenizer(`mytokenizer`) which basically performs `.split()` to create list of tokens. You can use your own tokenizer with appropriate sophistication.

In [37]:
import mytokenizer

In [38]:
from rouge_score import rouge_scorer

reference = ["f ( z ) = \sum _ { n = - \infty } ^ { \infty } f _ { n } e ^ { 2 \pi i n z }"]
candidate = [ "f ( z ) = \sum _ { n = - \infty } ^ { \in \{ \pi i n z } , f _ { z } e ^ { 2 0 d ^ { 3 } , 4 0 , 7 2 0 s } , , , \in ^ { 4 } , 7 , 9 2 0 n }"]

scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True, tokenizer = mytokenizer.DefaultTokenizer())
scores = scorer.score(reference[0], candidate[0])
for key in scores: # fmeasure is the ROUGE-N score
    print(f'{key}: {scores[key]}')

rouge1: Score(precision=0.47619047619047616, recall=0.967741935483871, fmeasure=0.6382978723404255)


### ROUGE-L:
Given two sequences $X=[x_1, x_2\ldots x_m]$ and $Y = [y_1, y_2, \ldots, y_n]$, $LCS(X,Y)$ is the length of the longest a subsequence common between $X$ and $Y$, where a subsequence of $X$ is obtained from $X$ by deleting some(possibly none) $x_i(s)$ but preserving the original order(same with $Y$). It is a known exercise in dynamic programming to calculate $LCS$ of two strings.

### Sentence-level LCS:
Assume that both the reference and the prediction/candidate are single sentences. (*Why this distinction between single and multiple-sentence text?* Well, sometimes each sentence is semantically complete and two consecutive sentences might not have any related semantics whatsoever). View a sentence as a sequence of *words*(in $\LaTeX$ code *words* for instance can be very naively considered as smallespiece of code delimited by white-spaces), so that each $x_i$ and $y_j$ in our definition is now a *word*

We now define the $LCS$ based precion and recall scores as $P_{lcs} = \dfrac{LCS(X,Y)}{n}$ and $R_{lcs} = \dfrac{LCS(X,Y)}{m}$, where the length of sequence $X$(the reference) is $m$ and that of $Y$(the candidate) is $n$. We defined the weighted F-score as: ROUGE-L($X$,$Y$) :=  $\dfrac{(1+\beta^2)R_{lcs}P_{lcs}}{\beta^2P_{lcs}+R_{lcs}}$ ,where the normal F-score is obtained when $\beta=1$(this one is implemented in python), select high $\beta>1$ for more weightage to $R_{lcs}$ and low $\beta<1$ for more weightage to $P_{lcs}$.

ROUGE-L has two obvious benefits over ROUGE-$N$:
+ It doesn't require consecutive matches but *in-sequence* matches( so, no n-gram analysis needed)
+ it automatically includes longest in-sequence common n-grams

In [39]:
# ROUGE-L on the previous example
reference = ["f ( z ) = \sum _ { n = - \infty } ^ { \infty } f _ { n } e ^ { 2 \pi i n z }"]
candidate = [ "f ( z ) = \sum _ { n = - \infty } ^ { \in \{ \pi i n z } , f _ { z } e ^ { 2 0 d ^ { 3 } , 4 0 , 7 2 0 s } , , , \in ^ { 4 } , 7 , 9 2 0 n }"]

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True, tokenizer = mytokenizer.DefaultTokenizer())
scores = scorer.score(reference[0], candidate[0])
for key in scores: # fmeasure is the ROUGE-L score
    print(f'{key}: {scores[key]}')
    

rougeL: Score(precision=0.4126984126984127, recall=0.8387096774193549, fmeasure=0.553191489361702)


See below one of the examples of a situation where ROUGE-L differentiates between two candidates of varying quality which ROUGE-$N$ (here $N=2$) cannot

In [40]:
reference = "police killed the gunman"
candidate1 = "police kill the gunman"
candidate2 = "the gunman kill police"

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True, tokenizer = mytokenizer.DefaultTokenizer())
scores = scorer.score(reference, candidate1)
print("For candidate1:")
for key in scores: # fmeasure is the ROUGE-L score
    print(f'{key}: {scores[key]}')
scores = scorer.score(reference, candidate2)
print("For candidate2:")
for key in scores: # fmeasure is the ROUGE-L score
    print(f'{key}: {scores[key]}')

print()

# ROUGE-2 can't differentiate the two candidates

scorer = rouge_scorer.RougeScorer(['rouge2'], use_stemmer=True, tokenizer = mytokenizer.DefaultTokenizer())
scores = scorer.score(reference, candidate1)
print("For candidate1:")
for key in scores: # fmeasure is the ROUGE-L score
    print(f'{key}: {scores[key]}')
scores = scorer.score(reference, candidate2)
print("For candidate2:")
for key in scores: # fmeasure is the ROUGE-L score
    print(f'{key}: {scores[key]}')

For candidate1:
rougeL: Score(precision=0.75, recall=0.75, fmeasure=0.75)
For candidate2:
rougeL: Score(precision=0.5, recall=0.5, fmeasure=0.5)

For candidate1:
rouge2: Score(precision=0.3333333333333333, recall=0.3333333333333333, fmeasure=0.3333333333333333)
For candidate2:
rouge2: Score(precision=0.3333333333333333, recall=0.3333333333333333, fmeasure=0.3333333333333333)


### Summary-Level LCS:

Now, instead of being single sentences, lets say both the candidate the reference are composed of multiple sentences.
Say, we have reference $R=[r_1,r_2,\ldots , r_u]$ of $u$ sentences containing $m$ total words and a candidate $C$ of $v$ sentences containing $n$ total words. Then we calculate the $LCS$ based precision and recall in this case in the following way:

$R_{lcs}=\dfrac{\sum\limits_{i=1}^u LCS_{\cup}(r_i,C)}{m}$ and $P_{lcs}=\dfrac{\sum\limits_{i=1}^u LCS_{\cup}(r_i,C)}{n}$, where
$LCS_{\cup}(r_i,C)$ is the length of the union of $LCSs$ between each $c_j \in C$ and $r_i$.
Let me illustrate this by an example: say $C=[c_1,c_2]$, where $c_1 = w_1 w_3 w_8 w_9 w_5$ and $c_2=w_1w_2w_6w_7w_8$ (each $w_k$ is a word), and let $r_i=w_1w_2w_3w_4w_5$. We have $lcs$ of $r_i$ and $c_1$ as $w_1w_3w_5$ and $lcs$ of $r_i$ and $c_2$ as $w_1w_2$, therefore the *union lcs* of $r_i$ and $C$ is $w_1w_2w_3w_5$ and $LCS_{\cup}(r_i,C)=4$.

Now the summary-level ROUGE-L score is defined as the weighted *F-measure* of $P_{lcs}$ and $R_{lcs}$, i.e. $ROUGE-L(C,R):=F_{lcs} = \dfrac{(1+\beta^2)R_{lcs}P_{lcs}}{\beta^2P_{lcs}+R_{lcs}}$ . Significance of $\beta$ is same as in ROUGE-$N$.

For multiple references, we follow a jackknifing procedure as earlier!

In [41]:
reference = "f ( z ) = \sum _ { n = - \infty } ^ { \infty } f _ { n } e ^ { 2 \pi i n z }. \varepsilon ^ { \mu \nu \alpha \beta } \partial _ { \nu } B _ { \alpha \beta } = 0 ."
candidate =  "f ( z ) = \sum _ { n = - \infty } ^ { \in \{ \pi i n z } , f _ { z } e ^ { 2 0 d ^ { 3 } , 4 0 , 7 2 0 s } , , , \in ^ { 4 } , 7 , 9 2 0 n }.\varepsilon ^ { 2 3 s \n u \alpha \beta } \partial _ { \nu } B _ { \alpha \beta } = 0 ."

scorer = rouge_scorer.RougeScorer(['rouge2'], use_stemmer=True,split_summaries=True, tokenizer = mytokenizer.DefaultTokenizer())
scores = scorer.score(reference, candidate)
for key in scores: # fmeasure is the ROUGE-L score
    print(f'{key}: {scores[key]}')


rouge2: Score(precision=0.5, recall=0.8269230769230769, fmeasure=0.6231884057971014)


### ROUGE-W:

The $LCS$ base *F-measure* is quite good, but there is also an inherent difficulty that it doesn't distinguish between *spatially* different *LCSs*. For e.g. let $X = [A,B,C,D,E,F,G]$ be a reference and $Y_1=[A,B,C,D,H,I,K]$ and $Y_2=[A,H,B,K,C,I,D]$ be two candidates. We have $LCS(X,Y_1)=LCS(X,Y_2)=4$ and $n=m=7$, hence both have same *ROUGE-L* scores, but its obvious that $Y_1$ is more favorable. To improve the basic LCS method, we can simply remember the length of consecutive matches encountered so far to a regular two dimensional dynamic program table computing LCS, update weight according to a new (consecutive)match and give more weight to larger consecutive matches. Finally the WLCS is the sum of weights given to each patch of consecutive matches between the reference and candidate sentence.

We use the following dynamic programming algorithm to calculate *WLCS(Weighted LCS)* between two sentences $X$ and $Y$

In [42]:

def wlcs(x, y, weight=None):
    m=len(x)
    n=len(y)
    c=[[0 for x in range(n)] for x in range(m)]
    w = [[0 for x in range(n)] for x in range(m)]
    for i in range(0,m):
        for j in range(0,n):
            # initialize the tables
            c[i][j]=0 # c is the DP table that computes the WLCS score
            w[i][j]=0
    for i in range(0,m):
        for j in range(0,n):
            if (x[i]==y[j]):
                k = w[i-1][j-1] # length of current number of consecutive matches ending at position i-1 and j-1
                update = func(k+1,weight)-func(k,weight) # update the current weight to the consecutive matches by f(k+1)-f(k), where f is an easily invertible function we choose 
                c[i][j] = c[i-1][j-1] + update
                w[i][j] = k + 1 # remember the length of consecutive matches at position i and j
            else:
                c[i][j] = max(c[i-1][j],c[i][j-1])
                w[i][j] = 0 # no match at position i and j, so length of current number of consecutive matches ending at these position is 0!
    return c[m-1][n-1] # the WLCS of x and y



Note that the weighting function *$f$(or func)* must have the property that $f(x+y) > f(x) + f(y)$ for any positive integers $x$
and $y$. In other words, consecutive matches are awarded more scores than non-consecutive matches. We also require $f$ to be easily invertible (we generally choose functions of the form $f(x)=x^{weight}, weight>0$. Then we define the *WLCS* based precision and recall as (Let $X$ be the reference and $Y$ be the candidate):

$P_{lcs}= f^{-1}( \dfrac{WLCS(X,Y)}{f(n)})$ and $R_{lcs}= f^{-1}( \dfrac{WLCS(X,Y)}{f(m)})$

Finally, *ROUGE-W* score is defined as the weighted *F-measure* of $P_{lcs}$ and $R_{lcs}$, i.e. $ROUGE-W(X,Y):=F_{lcs} = \dfrac{(1+\beta^2)R_{lcs}P_{lcs}}{\beta^2P_{lcs}+R_{lcs}}$ 

In [43]:
import math
def func(x,weight,inverse=False):
    if inverse:
        return math.pow(x,1/weight)
    else:
        return math.pow(x,weight)
    
def rougew(x,y,weight=None):
    m=len(x)
    n=len(y)
    wlcs1 = wlcs(x,y,weight=weight)
    P = func(wlcs1/func(n,weight),weight=weight, inverse=True)
    R = func(wlcs1/func(m,weight),weight=weight, inverse=True)
    return (2*P*R/(P+R))


reference="A B C D E F G".split()
candidate1 = "A B C D H I K".split()
candidate2 = "A H B K C I D".split()

# ROUGE-W clearly distinguishes the two candidates!
print("WLCS of candidate 1 w.r.t. reference is: ",wlcs(reference,candidate1,weight=2))
print("ROUGE-W score of candidate 1 is: ",rougew(reference,candidate1,weight=2))
print()
print("WLCS of candidate 1 w.r.t. reference is: ",wlcs(reference,candidate2,weight=2))
print("ROUGE-W score of candidate 1 is: ",rougew(reference,candidate2,weight=2))

WLCS of candidate 1 w.r.t. reference is:  16.0
ROUGE-W score of candidate 1 is:  0.5714285714285714

WLCS of candidate 1 w.r.t. reference is:  4.0
ROUGE-W score of candidate 1 is:  0.2857142857142857


## Classification related
### Accuracy

<mark> Definition (*Confusion Matrix*)</mark>:The columns of the matrix represent the instances of the predicted classes predicted by a classifier and the rows represent the instances of the actual class. It is used to visualize the performance of a classifier. Note that the role of rows and columns can be inverted in some representations.

Let us work with a binary classifier for now, with two classes namely **Positive** and **Negative**.

\begin{array}{|c|c|c|}
\hline
 & \textbf{Predicted Negative} & \textbf{Predicted Positive} \\
\hline
\textbf{Actual Negative} & TN & FP \\
\hline
\textbf{Actual Positive} & FN & TP \\
\hline
\end{array}

where $TN: True \ Negative$, $TP: True \ Positive$, $FN:False \ Negative$, $FP: False \ Positive$.

Then *Accuracy* is then defined as : $ = {{TP + TN} \over {TP + TN + FP + FN}}$
The following is a logistic regression model which uses artificial train and test set. <mark>You can ignore all the code and simply look at the confusion matrix to calculate accuracy.</mark>

In [44]:
# Import required libraries
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=100, n_features=2, n_informative=2, 
                            n_redundant=0, random_state=42, n_classes=2)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Logistic Regression model
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()  # Extract values from the confusion matrix

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Print results
print("Confusion Matrix:")
print(f"              Predicted Negative   Predicted Positive")
print(f"Actual Negative      {tn}                  {fp}")
print(f"Actual Positive      {fn}                  {tp}")

print("\nAccuracy of the classifier:")
print(f"Accuracy = {accuracy:.2%}")




Confusion Matrix:
              Predicted Negative   Predicted Positive
Actual Negative      15                  1
Actual Positive      0                  14

Accuracy of the classifier:
Accuracy = 96.67%


**From now on, we will cook up arbitary confusion matrices, just to illustrate the metric, without actually spawning the classifier!**

Note: Accuracy alone is not a sufficient metric of performance as shown by the following example:

\begin{array}{|c|c|c|}
\hline
 & \textbf{Predicted Negative} & \textbf{Predicted Positive} \\
\hline
\textbf{Actual Negative} & 0 & 5 \\
\hline
\textbf{Actual Positive} & 0 & 95 \\
\hline
\end{array}

The classfier can have $0.95$ accuracy predicting "*Positive*" always!

### Precision
<mark>Definition</mark>: Fraction of True positives to all predicted positives.
$$ precision={{TP} \over {TP + FP}}$$.

This roughly translates to the classifier's "confidence" in predicting positives--important where False positives are very costly.

<a id='two'></a>
### Recall
<mark> Definition</mark>: Fraction of True positives to actual positives.
$$Recall = {{TP} \over {TP + FN}}$$

Recall is crucial in applications where missing positive cases (False Negatives) has serious consequences: Medical diagnosis, Disaster alerts, etc.

### F1-Score
<mark>Definition</mark>: Harmonic mean of precision and recall.
$$F1-score = 2*\frac{Precision*Recall}{Precision+Recall}$$

Precision focuses on the quality of positive predictions (How many of the predicted positives are correct?), whereas Recall focuses on the quantity of true positives identified (How many of the actual positives were found?).
Together, they provide a fuller picture of a classifier's performance and are often combined using the F1-score.



Main Credits @ https://python-course.eu/machine-learning/evaluation-metrics.php and @ https://dl.acm.org/doi/10.3115/1073083.1073135 